In [1]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from torch.utils.data import DataLoader, Dataset
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd
import json

input_file_path = r"D:\Sharif University of Tech\Data\Library Recommender\Pypi data\OriginalItems.json"
with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)


data = pd.DataFrame(data)

Tokenization :

In [2]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

class LibraryDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [3]:
data

,Package,Command,Release,Summary,Score_link,License,Homepage,Description,url,project,spider,server,date,Topic,tags
0,[pyfldm 1.0.1],[pip install pyfldm],"[Sep 3, 2023]",[A library for controlling Fldigi via xmlrpc],[https://libraries.io/pypi/pyfldm],[OSI Approved :: GNU Lesser General Public Lic...,[https://github.com/philliphall131/pyfldm],[ PyFLDM Python Fast Light Digital Modem A l...,[https://pypi.org/project/pyfldm/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 12:49:55],NaN,NaN
1,[bitsnbytes 1.0.0],[pip install bitsnbytes],"[Jul 12, 2021]","[Introduces the Bit, Nibble, Byte and ByteLike...",[https://libraries.io/pypi/bitsnbytes],[OSI Approved :: MIT License],NaN,[],[https://pypi.org/project/bitsnbytes/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 12:49:55],NaN,NaN
2,[fcards 0.3.1],[pip install fcards],"[Sep 24, 2019]",[Digital flashcards],[https://libraries.io/pypi/fcards],[OSI Approved :: MIT License],[https://github.com/pypa/sampleproject],"[ This package allows you to create, edit and ...",[https://pypi.org/project/fcards/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 12:49:55],NaN,NaN
3,[lintpy 0.1],[pip install lintpy],"[Dec 31, 2017]",[The funniest joke in the world],[https://libraries.io/pypi/lintpy],NaN,[http://github.com/abhinavralhan/lintpy],[],[https://pypi.org/project/lintpy/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 12:49:55],NaN,NaN
4,[django-cms-light 0.0.2],[pip install django-cms-light],"[Jul 19, 2016]",[Django CMS Light provides internet as a servi...,[https://libraries.io/pypi/django-cms-light],[OSI Approved :: MIT License],[http://dcl-cms.com],[ UNKNOWN ],[https://pypi.org/project/django-cms-light/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 12:49:55],[Internet :: WWW/HTTP],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9692,[odoo12-addon-helpdesk-mgmt-project 12.0.1.1.1],[pip install odoo12-addon-helpdesk-mgmt-project],"[Apr 16, 2021]",[Add the option to select project in the ticke...,[https://libraries.io/pypi/odoo12-addon-helpde...,[OSI Approved :: GNU Affero General Public Lic...,[https://github.com/OCA/helpdesk],[ This module adds Project in Helpdesk.We...,[https://pypi.org/project/odoo12-addon-helpdes...,[SpySnake],[basic],[equanta-1-0],[2024-02-03 13:04:29],NaN,NaN
9693,[helloworldpkg 0.0.3],[pip install helloworldpkg],"[Jan 17, 2023]",[Just hello world],[https://libraries.io/pypi/helloworldpkg],NaN,NaN,[],[https://pypi.org/project/helloworldpkg/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 13:04:29],NaN,[python]
9694,[DmaPlotter 0.1.7],[pip install DmaPlotter],"[Dec 9, 2020]",[A one-function tool for visualizing metrics b...,[https://libraries.io/pypi/DmaPlotter],[OSI Approved :: MIT License],NaN,[],[https://pypi.org/project/DmaPlotter/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 13:04:29],NaN,NaN
9695,[my_chrome_bookmarks 1.0.2],[pip install my_chrome_bookmarks],"[Jan 8, 2022]",[Utils to load local chrome bookmarks.],[https://libraries.io/pypi/my_chrome_bookmarks],[OSI Approved :: Apache Software License],[https://github.com/conchylicultor/my-chrome-b...,[ my-chrome-bookmarks Python extension to re...,[https://pypi.org/project/my_chrome_bookmarks/],[SpySnake],[basic],[equanta-1-0],[2024-02-03 13:04:29],[Software Development :: Libraries :: Python M...,"[google,]"


Encoding the data :

In [4]:
X = list(data['Description'])
y = list(data['library'].factorize()[0])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=512)

train_dataset = LibraryDataset(train_encodings, y_train)
val_dataset = LibraryDataset(val_encodings, y_val)

KeyError: 'library'

The model would be :

In [ ]:
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = len(set(y))
model = RobertaForSequenceClassification(config)

Now lets start training :

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

optim = optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
for epoch in range(3):
    model.train()
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")